In [2]:
import importlib.util
from pathlib import Path

# load module from a path because package/folder name contains a hyphen which is invalid in import statements
module_path = Path("backend-dinesh/ml/preprocessing/select_series.py")
spec = importlib.util.spec_from_file_location("select_series", module_path)
select_series = importlib.util.module_from_spec(spec)
spec.loader.exec_module(select_series)

find_main_ct_series = select_series.find_main_ct_series

folder = "LIDC-IDRI/manifest-1764432147192/LIDC-IDRI/LIDC-IDRI-0001"
series, count = find_main_ct_series(folder)
print(series, count)


LIDC-IDRI/manifest-1764432147192/LIDC-IDRI/LIDC-IDRI-0001\01-01-2000-NA-NA-30178\3000566.000000-NA-03192 133


In [3]:
# load the load_dicom module from the same preprocessing folder as select_series.py
load_dicom_path = module_path.parent / "load_dicom.py"
spec_ld = importlib.util.spec_from_file_location("ml.preprocessing.load_dicom", load_dicom_path)
load_dicom = importlib.util.module_from_spec(spec_ld)
spec_ld.loader.exec_module(load_dicom)

vol, spacing = load_dicom.load_dicom_series(series)
print(vol.shape, spacing)


(133, 512, 512) [2.5, 0.703125, 0.703125]


In [6]:
resample_path = module_path.parent / "resample.py"
spec_rs = importlib.util.spec_from_file_location("ml.preprocessing.resample", resample_path)
resample = importlib.util.module_from_spec(spec_rs)
spec_rs.loader.exec_module(resample)
resampled_vol, new_spacing = resample.resample_to_iso(vol, spacing)
print(resampled_vol.shape, new_spacing)


(360, 360, 332) [1.0, 1.0, 1.0]


In [10]:
lung_segmentation_path = module_path.parent / "lung_segmentation.py"
spec_ls = importlib.util.spec_from_file_location("ml.preprocessing.lung_segmentation", lung_segmentation_path)
lung_segmentation = importlib.util.module_from_spec(spec_ls)
spec_ls.loader.exec_module(lung_segmentation)
lung_mask = lung_segmentation.segment_lungs(resampled_vol)
print(lung_mask.shape)


Downloading: "https://github.com/JoHof/lungmask/releases/download/v0.0/unet_r231-d5d2fc3d.pth" to C:\Users\padal/.cache\torch\hub\checkpoints\unet_r231-d5d2fc3d.pth


100%|██████████| 119M/119M [00:27<00:00, 4.60MB/s] 
100%|██████████| 18/18.0 [00:05<00:00,  3.16it/s]

lungmask 2025-11-30 00:14:58 Postprocessing



100%|██████████| 3/3 [00:00<00:00, 56.27it/s]


(360, 360, 332)


In [12]:
# try the preprocessing folder first, then the detection folder sibling
log_detector_path = module_path.parent / "log_detector.py"
if not log_detector_path.exists():
	alt_path = module_path.parent.parent / "detection" / "log_detector.py"
	if alt_path.exists():
		log_detector_path = alt_path
	else:
		raise FileNotFoundError(
			f"log_detector.py not found at {module_path.parent} or {alt_path}. "
			"Please ensure the file exists or update the path."
		)

spec_ld = importlib.util.spec_from_file_location("ml.detection.log_detector", log_detector_path)
log_detector = importlib.util.module_from_spec(spec_ld)
spec_ld.loader.exec_module(log_detector)
cands, logmap = log_detector.log_nodule_candidates(resampled_vol, lung_mask)
print("Candidates found:", len(cands))
print(cands[:10])


Candidates found: 37751402
[(np.int64(0), np.int64(0), np.int64(0)), (np.int64(0), np.int64(0), np.int64(1)), (np.int64(0), np.int64(0), np.int64(2)), (np.int64(0), np.int64(0), np.int64(3)), (np.int64(0), np.int64(0), np.int64(4)), (np.int64(0), np.int64(0), np.int64(5)), (np.int64(0), np.int64(0), np.int64(6)), (np.int64(0), np.int64(0), np.int64(7)), (np.int64(0), np.int64(0), np.int64(8)), (np.int64(0), np.int64(0), np.int64(9))]


In [13]:
filter_candidates_path = module_path.parent / "filter_candidates.py"
if not filter_candidates_path.exists():
	alt_path = module_path.parent.parent / "detection" / "filter_candidates.py"
	if alt_path.exists():
		filter_candidates_path = alt_path
	else:
		raise FileNotFoundError(
			f"filter_candidates.py not found at {module_path.parent} or {alt_path}. "
			"Please ensure the file exists or update the path."
		)
spec_ld = importlib.util.spec_from_file_location("ml.detection.filter_candidates", filter_candidates_path)
filter_candidates = importlib.util.module_from_spec(spec_ld)
spec_ld.loader.exec_module(filter_candidates)
  
filtered = filter_candidates.filter_candidates(cands, resampled_vol, lung_mask)
print("Filtered:", len(filtered))
print(filtered[:10])


Filtered: 3522
[(np.int64(60), np.int64(244), np.int64(104)), (np.int64(62), np.int64(245), np.int64(109)), (np.int64(64), np.int64(246), np.int64(113)), (np.int64(65), np.int64(239), np.int64(91)), (np.int64(65), np.int64(247), np.int64(118)), (np.int64(69), np.int64(245), np.int64(131)), (np.int64(70), np.int64(244), np.int64(107)), (np.int64(70), np.int64(246), np.int64(114)), (np.int64(70), np.int64(246), np.int64(118)), (np.int64(71), np.int64(240), np.int64(97))]


In [14]:
patch_extractor_path = module_path.parent / "patch_extractor.py"
if not patch_extractor_path.exists():
	alt_path = module_path.parent.parent / "detection" / "patch_extractor.py"
	if alt_path.exists():
		patch_extractor_path = alt_path
	else:
		raise FileNotFoundError(
			f"patch_extractor.py not found at {module_path.parent} or {alt_path}. "
			"Please ensure the file exists or update the path."
		)
spec_ld = importlib.util.spec_from_file_location("ml.detection.patch_extractor", patch_extractor_path)
patch_extractor = importlib.util.module_from_spec(spec_ld)
spec_ld.loader.exec_module(patch_extractor)

patch = patch_extractor.extract_patch(resampled_vol, filtered[0])
print(patch.shape)


(32, 32, 32)


In [22]:
# load feature extractor and postprocess modules directly from files (package import may fail due to folder names)
feature_extractor_path = module_path.parent.parent / "features" / "feature_extractor.py"
if not feature_extractor_path.exists():
    raise FileNotFoundError(f"{feature_extractor_path} not found. Please check the path.")

spec_fe = importlib.util.spec_from_file_location("ml.features.feature_extractor", feature_extractor_path)
feature_extractor = importlib.util.module_from_spec(spec_fe)
spec_fe.loader.exec_module(feature_extractor)
compute_hu_stats = feature_extractor.compute_hu_stats
compute_long_axis_mm = feature_extractor.compute_long_axis_mm
compute_volume_mm3 = feature_extractor.compute_volume_mm3

classify_type_path = module_path.parent.parent / "postprocess" / "classify_type.py"
if not classify_type_path.exists():
    raise FileNotFoundError(f"{classify_type_path} not found. Please check the path.")

spec_ct = importlib.util.spec_from_file_location("ml.postprocess.classify_type", classify_type_path)
classify_type_mod = importlib.util.module_from_spec(spec_ct)
spec_ct.loader.exec_module(classify_type_mod)
classify_nodule_type = classify_type_mod.classify_nodule_type

classify_lobe_path = module_path.parent.parent / "postprocess" / "classify_lobe.py"
if not classify_lobe_path.exists():
    raise FileNotFoundError(f"{classify_lobe_path} not found. Please check the path.")

spec_cl = importlib.util.spec_from_file_location("ml.postprocess.classify_lobe", classify_lobe_path)
classify_lobe_mod = importlib.util.module_from_spec(spec_cl)
spec_cl.loader.exec_module(classify_lobe_mod)
classify_lobe = classify_lobe_mod.classify_lobe

features = []

for center in filtered:
    patch = patch_extractor.extract_patch(resampled_vol, center)

    # intensity
    hu_mean, hu_std = compute_hu_stats(patch)

    # morphology
    long_axis = compute_long_axis_mm(patch, spacing=[1,1,1])
    volume_mm3 = compute_volume_mm3(patch, spacing=[1,1,1])

    # type
    nodule_type = classify_nodule_type(hu_mean)

    # location
    lobe = classify_lobe(center, resampled_vol.shape)

    features.append({
        "center": center,
        "hu_mean": hu_mean,
        "hu_std": hu_std,
        "long_axis_mm": long_axis,
        "volume_mm3": volume_mm3,
        "type": nodule_type,
        "lobe": lobe
    })

len(features)


3522

In [23]:
import csv
import os

study_id = "LIDC-IDRI-0001"
save_dir = module_path.parent.parent.parent / "data" / "features"
save_dir.mkdir(parents=True, exist_ok=True)

csv_path = save_dir / f"{study_id}_features.csv"

with open(csv_path, "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["center_z", "center_y", "center_x",
                     "hu_mean", "hu_std",
                     "long_axis_mm", "volume_mm3",
                     "type", "lobe"])
    
    for ft in features:
        cz, cy, cx = ft["center"]
        writer.writerow([
            int(cz), int(cy), int(cx),
            ft["hu_mean"], ft["hu_std"],
            ft["long_axis_mm"], ft["volume_mm3"],
            ft["type"], ft["lobe"]
        ])

csv_path


WindowsPath('backend-dinesh/data/features/LIDC-IDRI-0001_features.csv')

In [24]:
from pathlib import Path
mlp_path = module_path.parent.parent / "risk" / "train_mlp.py"

spec_mlp = importlib.util.spec_from_file_location("ml.risk.train_mlp", mlp_path)
mlp_mod = importlib.util.module_from_spec(spec_mlp)
spec_mlp.loader.exec_module(mlp_mod)

csv_dir = module_path.parent.parent.parent / "data" / "features"
csv_files = list(csv_dir.glob("*_features.csv"))

save_dir = module_path.parent.parent.parent / "models" / "risk_head"
model = mlp_mod.train_risk_mlp(csv_files, save_dir)


Epoch 1/10 - Loss: 0.1402
Epoch 2/10 - Loss: 0.0108
Epoch 3/10 - Loss: 0.0199
Epoch 4/10 - Loss: 0.0043
Epoch 5/10 - Loss: 0.0106
Epoch 6/10 - Loss: 0.0019
Epoch 7/10 - Loss: 0.0016
Epoch 8/10 - Loss: 0.0001
Epoch 9/10 - Loss: 0.0008
Epoch 10/10 - Loss: 0.0052
Model saved: backend-dinesh\models\risk_head\risk_head.pth


In [25]:
risk_path = module_path.parent.parent / "risk" / "predict_risk.py"
spec_r = importlib.util.spec_from_file_location("ml.risk.predict_risk", risk_path)
risk_mod = importlib.util.module_from_spec(spec_r)
spec_r.loader.exec_module(risk_mod)

model_dir = module_path.parent.parent.parent / "models" / "risk_head"
risk_model = risk_mod.RiskHead(model_dir / "risk_head.pth", model_dir / "risk_scaler.pkl")

# Test on first feature
ft = features[0]
vals = [ft["hu_mean"], ft["hu_std"], ft["long_axis_mm"], ft["volume_mm3"]]

p = risk_model.predict(vals)
p_mc, ent = risk_model.predict_mc_dropout(vals, T=5)

print("Risk:", p)
print("MC Risk:", p_mc)
print("Entropy:", ent)


Risk: 1.0
MC Risk: 1.0
Entropy: -9.999999505838704e-08


In [21]:
study_id = "LIDC-IDRI-0001"

mal_scores = []
uncs = []

for ft in features:
    vals = [ft["hu_mean"], ft["hu_std"], ft["long_axis_mm"], ft["volume_mm3"]]

    p = risk_model.predict(vals)
    p_mc, ent = risk_model.predict_mc_dropout(vals, T=5)

    mal_scores.append(p)

    uncs.append({
        "confidence": p_mc,
        "entropy": ent,
        "needs_review": ent > 0.4
    })

builder_path = module_path.parent.parent / "json_builder" / "builder.py"
spec_b = importlib.util.spec_from_file_location("ml.json_builder.builder", builder_path)
builder_mod = importlib.util.module_from_spec(spec_b)
spec_b.loader.exec_module(builder_mod)

save_json = module_path.parent.parent.parent / "outputs" / f"{study_id}_findings.json"
save_json.parent.mkdir(parents=True, exist_ok=True)

builder_mod.build_findings_json(
    study_id,
    spacing=[1,1,1],
    volume_shape=resampled_vol.shape,
    filtered_candidates=filtered,
    features=features,
    malignancy_scores=mal_scores,
    uncertainties=uncs,
    output_path=str(save_json)
)

save_json


WindowsPath('backend-dinesh/outputs/LIDC-IDRI-0001_findings.json')

In [27]:
# Load module from file
lndb_extract_path = module_path.parent.parent / "lndb" / "extract_lndb_features.py"

spec_ex = importlib.util.spec_from_file_location("backend-dinesh.ml.lndb.extract", lndb_extract_path)
lndb_extract = importlib.util.module_from_spec(spec_ex)
spec_ex.loader.exec_module(lndb_extract)

# Run extraction
lndb_root = Path("LNDBv4")
save_csv = module_path.parent.parent.parent / "data" / "risk_features" / "lndb_features.csv"
save_csv.parent.mkdir(parents=True, exist_ok=True)

lndb_extract.extract_lndb_features(str(lndb_root), str(save_csv))


KeyError: 'NoduleID'

In [29]:
import pandas as pd

df = pd.read_csv("LNDBv4/allNods.csv")
print(df.columns)
df.head()


Index(['LNDbID', 'RadID', 'RadFinding', 'FindingID', 'Nodule', 'x', 'y', 'z',
       'DiamEq_Rad', 'Texture', 'Calcification', 'InternalStructure',
       'Lobulation', 'Malignancy', 'Margin', 'Sphericity', 'Spiculation',
       'Subtlety', 'Lobe', 'TextInstanceID', 'TextQuestion', 'Pos_Text',
       'Diam_Text', 'NodType', 'Caract_Text', 'Where'],
      dtype='object')


,LNDbID,RadID,RadFinding,FindingID,Nodule,x,y,z,DiamEq_Rad,Texture,...,Spiculation,Subtlety,Lobe,TextInstanceID,TextQuestion,Pos_Text,Diam_Text,NodType,Caract_Text,Where
0,1,"1,2,3","1,1,1",1,"1,1,1",-44.203451,-119.073242,-37.5,7.516572,5.0,...,2.333333,3.666667,3,0.0,NaN,apical,8.5,nod,margin: 1,TextReport+RadAnnotation
1,1,1,2,2,1,25.852539,-126.969727,-45.5,6.626905,4.0,...,1.000000,3.000000,1,NaN,NaN,NaN,NaN,NaN,NaN,RadAnnotation
2,2,"1,2,3","1,1,3",1,"1,1,1",88.895508,-123.867513,-129.5,8.971132,5.0,...,1.000000,5.000000,1,0.0,how many?,NaN,NaN,nod,calcification: 5,TextReport+RadAnnotation
3,2,"1,3","2,2",2,"1,1",63.534180,-112.756836,-117.5,6.937737,5.0,...,1.000000,5.000000,1,0.0,how many?,NaN,NaN,nod,calcification: 5,TextReport+RadAnnotation
4,2,"1,3","3,5",3,"1,1",-103.850586,-116.742188,-253.0,8.284458,5.0,...,1.000000,3.500000,5,NaN,NaN,NaN,NaN,NaN,NaN,RadAnnotation


In [32]:
df_nod = pd.read_csv(z.open("trainNodules.csv"))
print("trainNodules columns:\n", df_nod.columns)
df_nod.head()


trainNodules columns:
 Index(['LNDbID', 'RadID', 'FindingID', 'x', 'y', 'z', 'Nodule', 'Volume',
       'Text'],
      dtype='object')


,LNDbID,RadID,FindingID,x,y,z,Nodule,Volume,Text
0,1,1,1,-44.608398,-119.073242,-37.5,1,440.908794,5
1,1,1,2,25.852539,-126.969727,-45.5,1,152.381031,4
2,1,2,1,-44.000977,-118.465820,-37.5,1,56.820045,5
3,1,3,1,-44.000977,-119.680664,-37.5,1,169.353252,5
4,2,1,1,88.895508,-123.625977,-129.5,1,339.187950,5


In [33]:
df_gt = pd.read_csv(z.open("trainNodules_gt.csv"))
print("\ntrainNodules_gt columns:\n", df_gt.columns)
df_gt.head()



trainNodules_gt columns:
 Index(['LNDbID', 'RadID', 'RadFindingID', 'FindingID', 'x', 'y', 'z',
       'AgrLevel', 'Nodule', 'Volume', 'Text'],
      dtype='object')


,LNDbID,RadID,RadFindingID,FindingID,x,y,z,AgrLevel,Nodule,Volume,Text
0,1,"1,2,3","1,1,1",1,-44.203451,-119.073242,-37.5,3,1,222.360697,5.0
1,1,1,2,2,25.852539,-126.969727,-45.5,1,1,152.381031,4.0
2,2,"1,2,3","1,1,3",1,88.895508,-123.867513,-129.5,3,1,378.042297,5.0
3,2,"1,3","2,2",2,63.534180,-112.756836,-117.5,2,1,174.844563,5.0
4,2,"1,3","3,5",3,-103.850586,-116.742188,-253.0,2,1,297.708309,5.0


In [34]:
df_nod = pd.read_csv(z.open("trainNodules.csv"))
print("trainNodules columns:", df_nod.columns)
df_nod.head()

df_gt = pd.read_csv(z.open("trainNodules_gt.csv"))
print("\ntrainNodules_gt columns:", df_gt.columns)
df_gt.head()


trainNodules columns: Index(['LNDbID', 'RadID', 'FindingID', 'x', 'y', 'z', 'Nodule', 'Volume',
       'Text'],
      dtype='object')

trainNodules_gt columns: Index(['LNDbID', 'RadID', 'RadFindingID', 'FindingID', 'x', 'y', 'z',
       'AgrLevel', 'Nodule', 'Volume', 'Text'],
      dtype='object')


,LNDbID,RadID,RadFindingID,FindingID,x,y,z,AgrLevel,Nodule,Volume,Text
0,1,"1,2,3","1,1,1",1,-44.203451,-119.073242,-37.5,3,1,222.360697,5.0
1,1,1,2,2,25.852539,-126.969727,-45.5,1,1,152.381031,4.0
2,2,"1,2,3","1,1,3",1,88.895508,-123.867513,-129.5,3,1,378.042297,5.0
3,2,"1,3","2,2",2,63.534180,-112.756836,-117.5,2,1,174.844563,5.0
4,2,"1,3","3,5",3,-103.850586,-116.742188,-253.0,2,1,297.708309,5.0


In [36]:
spec = importlib.util.spec_from_file_location("lndb_fast", Path("backend-dinesh/ml/lndb/extract_lndb_fast.py"))
mod = importlib.util.module_from_spec(spec)
spec.loader.exec_module(mod)
mod.extract_lndb_fast("LNDBv4", "backend-dinesh/data/risk_features/lndb_features.csv", radius_mm=8.0)


Saved: backend-dinesh/data/risk_features/lndb_features.csv


In [37]:
spec = importlib.util.spec_from_file_location("train_lndb", Path("backend-dinesh/ml/risk/train_lndb_mlp.py"))
mod = importlib.util.module_from_spec(spec)
spec.loader.exec_module(mod)
mod.train_lndb_mlp("backend-dinesh/data/risk_features/lndb_features.csv", "backend-dinesh/models/risk_head", epochs=12)


Epoch 1/12 - Loss: 0.5904
Epoch 2/12 - Loss: 0.4712
Epoch 3/12 - Loss: 0.3451
Epoch 4/12 - Loss: 0.2499
Epoch 5/12 - Loss: 0.1899
Epoch 6/12 - Loss: 0.1751
Epoch 7/12 - Loss: 0.1665
Epoch 8/12 - Loss: 0.1591
Epoch 9/12 - Loss: 0.1910
Epoch 10/12 - Loss: 0.1651
Epoch 11/12 - Loss: 0.1605
Epoch 12/12 - Loss: 0.1648
Saved model & scaler to backend-dinesh\models\risk_head


Sequential(
  (0): Linear(in_features=4, out_features=32, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.2, inplace=False)
  (3): Linear(in_features=32, out_features=16, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.1, inplace=False)
  (6): Linear(in_features=16, out_features=1, bias=True)
  (7): Sigmoid()
)

In [41]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, roc_curve, brier_score_loss
from sklearn.preprocessing import StandardScaler
import joblib, torch

df = pd.read_csv("backend-dinesh/data/risk_features/lndb_features.csv")
X = df[["hu_mean","hu_std","long_axis_mm","volume_mm3"]].fillna(0).values.astype(np.float32)
y = df["malignancy"].astype(int).values
scaler = joblib.load("backend-dinesh/models/risk_head/risk_scaler.pkl")
model_state = torch.load("backend-dinesh/models/risk_head/risk_head.pth", map_location="cpu")

# recreate model
import torch.nn as nn
model = nn.Sequential(nn.Linear(4,32), nn.ReLU(), nn.Dropout(0.2), nn.Linear(32,16), nn.ReLU(), nn.Dropout(0.1), nn.Linear(16,1), nn.Sigmoid())
model.load_state_dict(model_state)
model.eval()

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
aucs = []
for train_idx, test_idx in skf.split(X,y):
    Xtr, Xte = scaler.transform(X[train_idx]), scaler.transform(X[test_idx])
    with torch.no_grad():
        preds = model(torch.tensor(Xte, dtype=torch.float32)).numpy().ravel()
    aucs.append(roc_auc_score(y[test_idx], preds))
print("5-fold AUCs:", aucs, "mean AUC:", np.mean(aucs))

# save ROC of last fold
fpr, tpr, _ = roc_curve(y[test_idx], preds)
plt.figure(); plt.plot(fpr,tpr); plt.xlabel("FPR"); plt.ylabel("TPR"); plt.title("ROC"); plt.grid(True)
out_dir = Path("outputs/metrics")
out_dir.mkdir(parents=True, exist_ok=True)
plt.savefig(str(out_dir / "roc_lndb.png"), bbox_inches="tight")
plt.close()

# calibration curve (binned)
from sklearn.calibration import calibration_curve
prob_true, prob_pred = calibration_curve(y[test_idx], preds, n_bins=10)
plt.figure(); plt.plot(prob_pred, prob_true, marker='o'); plt.plot([0,1],[0,1], '--'); plt.xlabel("Predicted"); plt.ylabel("Observed"); plt.title("Calibration"); plt.grid(True)
plt.savefig(str(out_dir / "calibration_lndb.png"), bbox_inches="tight")
plt.close()
print("Saved ROC & calibration to outputs/metrics/")


5-fold AUCs: [0.5853427895981087, 0.5513002364066193, 0.5200945626477541, 0.5702127659574469, 0.5101063829787233] mean AUC: 0.5474113475177305
Saved ROC & calibration to outputs/metrics/
Saved ROC & calibration to outputs/metrics/


In [42]:
import joblib, torch, numpy as np, pandas as pd
from pathlib import Path
scaler = joblib.load("backend-dinesh/models/risk_head/risk_scaler.pkl")
state = torch.load("backend-dinesh/models/risk_head/risk_head.pth", map_location="cpu")
import torch.nn as nn
model = nn.Sequential(nn.Linear(4,32), nn.ReLU(), nn.Dropout(0.2), nn.Linear(32,16), nn.ReLU(), nn.Dropout(0.1), nn.Linear(16,1), nn.Sigmoid())
model.load_state_dict(state); model.eval()

def predict_prob(feat_row, T=8):
    x = np.array([feat_row["hu_mean"], feat_row["hu_std"], feat_row["long_axis_mm"], feat_row["volume_mm3"]], dtype=np.float32).reshape(1,-1)
    x_s = scaler.transform(x)
    x_t = torch.tensor(x_s, dtype=torch.float32)
    # deterministic
    with torch.no_grad():
        p = model(x_t).item()
    # MC-dropout (enable dropout layers)
    for m in model.modules():
        if isinstance(m, nn.Dropout): m.train()
    probs = []
    with torch.no_grad():
        for _ in range(T):
            probs.append(model(x_t).item())
    for m in model.modules():
        if isinstance(m, nn.Dropout): m.eval()
    probs = np.array(probs)
    return p, float(probs.mean()), float(-(probs.mean()*np.log(probs.mean()+1e-9) + (1-probs.mean())*np.log(1-probs.mean()+1e-9)))

# example: run on LIDC features file
df = pd.read_csv("backend-dinesh/data/features/LIDC-IDRI-0001_features.csv")
out = []
for _, r in df.iterrows():
    p_det, p_mc, ent = predict_prob(r, T=8)
    out.append({"prob":p_det, "mc":p_mc, "entropy":ent})
pd.DataFrame(out).head()


,prob,mc,entropy
0,0.0,0.0,-1.000000e-09
1,0.0,0.0,-1.000000e-09
2,0.0,0.0,-1.000000e-09
3,0.0,0.0,-1.000000e-09
4,0.0,0.0,-1.000000e-09


In [44]:
import json
j = json.load(open("backend-dinesh/outputs/LIDC-IDRI-0001_findings.json"))
# basic checks
print("study_id:", j.get("study_id"))
print("num_candidates:", j.get("num_candidates"))
print("num_nodules:", j.get("num_nodules"))
print("processing_time_seconds:", j.get("processing_time_seconds"))
print("lung fields:", j.get("lung_health"), j.get("emphysema_score"), j.get("consolidation_score"))
n0 = j["nodules"][0] if j["nodules"] else None
print("first nodule keys:", list(n0.keys()) if n0 else None)
print("first nodule prob:", n0.get("prob_malignant") if n0 else None)
print("first nodule uncertainty:", n0.get("uncertainty") if n0 else None)
print("first nodule location field:", n0.get("location"), "lobe:", n0.get("lobe"))
# quick sanity: long axis vs volume
if n0:
    la = n0.get("long_axis_mm",0)
    vol = n0.get("volume_mm3",0)
    print("sanity ratio long_axis_mm(mm)^3 vs volume_mm3:", la**3, vol)


study_id: LIDC-IDRI-0001
num_candidates: 93
num_nodules: 93
processing_time_seconds: 6.125739574432373
lung fields: Lungs appear within expected attenuation ranges. 0.0010729492042243047 0.9141988788487282
first nodule keys: ['id', 'centroid', 'coordinates', 'bbox', 'mask_path', 'long_axis_mm', 'volume_mm3', 'type', 'lobe', 'location', 'prob_malignant', 'uncertainty']
first nodule prob: 0.95
first nodule uncertainty: {'confidence': 0.9945688275300512, 'entropy': 0.03374321933759373, 'needs_review': False}
first nodule location field: right middle lobe lobe: right middle lobe
sanity ratio long_axis_mm(mm)^3 vs volume_mm3: 78139.0522579655 32071.0


In [1]:
import json
j = json.load(open("backend-dinesh/outputs/LIDC-IDRI-0001_findings.json"))

# Key Phase-1 checks:

print("study_id:", j["study_id"])
print("num_nodules:", j["num_nodules"])

# 1) Check risk variation
risks = [n["prob_malignant"] for n in j["nodules"]]
print("risk min:", min(risks))
print("risk max:", max(risks))
print("unique risks:", len(set(round(r,3) for r in risks)))

# 2) Check type values
types = set(n["type"] for n in j["nodules"])
print("types:", types)

# 3) Check lobe values
lobes = set(n["location"] for n in j["nodules"])
print("lobes:", lobes)

# 4) Check uncertainty
unc = [n["uncertainty"]["needs_review"] for n in j["nodules"]]
print("needs_review count:", sum(unc))

# 5) Size-volume sanity
first = j["nodules"][0]
print(first["long_axis_mm"], first["volume_mm3"])


study_id: LIDC-IDRI-0001
num_nodules: 93
risk min: 0.9
risk max: 0.9
unique risks: 1
types: {'subsolid', 'solid', 'ground-glass'}
lobes: {'left lower lobe', 'left middle lobe', 'right middle lobe', 'right lower lobe', 'right upper lobe'}
needs_review count: 0
42.751961609058235 32071.0


In [2]:
import json
j = json.load(open("backend-dinesh/outputs/LIDC-IDRI-0001_findings.json"))

risks = [n["prob_malignant"] for n in j["nodules"]]
print(min(risks), max(risks), len(set(risks)))

print(set(n["type"] for n in j["nodules"]))
print(set(n["location"] for n in j["nodules"]))
print(sum(n["uncertainty"]["needs_review"] for n in j["nodules"]))


0.9 0.9 1
{'subsolid', 'solid', 'ground-glass'}
{'left lower lobe', 'left middle lobe', 'right middle lobe', 'right lower lobe', 'right upper lobe'}
0


In [1]:
import json
j = json.load(open("backend-dinesh/outputs/LIDC-IDRI-0001_findings.json"))

risks = [n["prob_malignant"] for n in j["nodules"]]
types = {n["type"] for n in j["nodules"]}
lobes = {n["location"] for n in j["nodules"]}
needs = [n["uncertainty"]["needs_review"] for n in j["nodules"]]

print("risk min:", min(risks))
print("risk max:", max(risks))
print("unique risks:", len(set([round(r,4) for r in risks])))

print("types:", types)
print("lobes:", lobes)
print("needs_review count:", sum(needs))

print("long_axis vs volume sample:",
      j["nodules"][0]["long_axis_mm"],
      j["nodules"][0]["volume_mm3"]
)


risk min: 0.9
risk max: 0.9
unique risks: 1
types: {'subsolid', 'solid', 'ground-glass'}
lobes: {'left lower lobe', 'right lower lobe', 'right middle lobe', 'right upper lobe', 'left lingula'}
needs_review count: 0
long_axis vs volume sample: 42.751961609058235 32071.0


In [2]:
import json
j = json.load(open("backend-dinesh/outputs/LIDC-IDRI-0001_findings.json"))
risks = [n["prob_malignant"] for n in j["nodules"]]
print("risk min:", min(risks))
print("risk max:", max(risks))
print("unique risks:", len(set([round(r,4) for r in risks])))
print("types:", set(n["type"] for n in j["nodules"]))
print("lobes:", set(n["location"] for n in j["nodules"]))
print("needs_review count:", sum(n["uncertainty"]["needs_review"] for n in j["nodules"]))
print("sample long_axis, vol:", j["nodules"][0]["long_axis_mm"], j["nodules"][0]["volume_mm3"])


risk min: 0.6447073954119117
risk max: 0.9
unique risks: 93
types: {'subsolid', 'solid', 'ground-glass'}
lobes: {'left lower lobe', 'right lower lobe', 'right middle lobe', 'right upper lobe', 'left lingula'}
needs_review count: 92
sample long_axis, vol: 42.751961609058235 32071.0


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_ANON_KEY = os.getenv("SUPABASE_ANON_KEY")
SUPABASE_SERVICE_KEY = os.getenv("SUPABASE_SERVICE_KEY")


In [4]:
import requests

BASE_URL = "http://127.0.0.1:8000"

headers = {
    "x-user-id": "5e5202af-44ad-4994-9515-ba2fb3eb4dec",
    "x-user-role": "patient"
}

zip_path = "test_ct.zip"

files = {
    "file": ("test_ct.zip", open(zip_path, "rb"), "application/zip")
}

resp = requests.post(f"{BASE_URL}/upload/scan", headers=headers, files=files)

print("Status:", resp.status_code)
try:
    print(resp.json())
except:
    print(resp.text)


Status: 200
{'case_id': '5097f192-e3f6-4514-8a77-9199983a750c', 'storage_path': '5097f192-e3f6-4514-8a77-9199983a750c/test_ct.zip'}


In [10]:
import requests

BASE_URL = "http://127.0.0.1:8000"

case_id = "5097f192-e3f6-4514-8a77-9199983a750c"

resp = requests.post(
    f"{BASE_URL}/process/case/{case_id}",
    headers={
        "x-user-id": "6255e461-2a14-4b3c-a635-de4428668fc0",  # operator
        "x-user-role": "operator"
    }
)

print("Status:", resp.status_code)
print(resp.text)


Status: 500
{"detail":"Pipeline error: Pipeline not found at: C:\\Users\\padal\\Documents\\Work\\FYP-1\\backend-dinesh\\backend-dinesh\\ml\\pipeline.py"}


In [11]:
resp = requests.post(
    "http://127.0.0.1:8000/process/case/5097f192-e3f6-4514-8a77-9199983a750c",
    headers={
        "x-user-id": "6255e461-2a14-4b3c-a635-de4428668fc0",
        "x-user-role": "operator"
    }
)
print(resp.status_code, resp.text)


200 {"status":"completed"}


In [12]:
resp = requests.post("http://127.0.0.1:8000/process/case/5097f192-e3f6-4514-8a77-9199983a750c",
                     headers={"x-user-id": "6255e461-2a14-4b3c-a635-de4428668fc0", "x-user-role":"operator"})
print(resp.status_code, resp.text)


200 {"status":"completed"}


In [19]:
import requests

resp = requests.post(
    "http://127.0.0.1:8000/process/case/5097f192-e3f6-4514-8a77-9199983a750c",
    headers={
        "x-user-id": "6255e461-2a14-4b3c-a635-de4428668fc0",  # operator
        "x-user-role": "operator"
    }
)

print("Status:", resp.status_code)
print(resp.text)


Status: 200
{"status":"completed"}


In [6]:
PATIENT_ID = "5e5202af-44ad-4994-9515-ba2fb3eb4dec"
OPERATOR_ID = "6255e461-2a14-4b3c-a635-de4428668fc0"
DOCTOR_ID   = "358f00bb-d2f3-4352-b9bd-29042d3e9c57"
FILE_PATH   = "test_ct_extracted.zip"
BACKEND_URL = "http://127.0.0.1:8000"


In [11]:
import requests

BASE = "http://127.0.0.1:8000"

patient_id = "5e5202af-44ad-4994-9515-ba2fb3eb4dec"
zip_path = "test_ct_extracted.zip"

headers = {
    "x-user-id": patient_id,
    "x-user-role": "patient",
}

# STEP 1 — upload
with open(zip_path, "rb") as f:
    r = requests.post(
        f"{BASE}/upload/scan",
        headers=headers,
        files={"file": ("test_ct.zip", f, "application/zip")}
    )

print("Status:", r.status_code)
print("Response:", r.text)


Status: 200
Response: {"case_id":"99d04c3e-c389-4dd3-9939-a78029f69886","storage_path":"99d04c3e-c389-4dd3-9939-a78029f69886/test_ct.zip"}


In [12]:
import requests

BASE = "http://127.0.0.1:8000"
operator_id = "6255e461-2a14-4b3c-a635-de4428668fc0"
case_id = "99d04c3e-c389-4dd3-9939-a78029f69886"

headers = {
    "x-user-id": operator_id,
    "x-user-role": "operator"
}

r = requests.post(f"{BASE}/process/case/{case_id}", headers=headers)
print("Status:", r.status_code)
print(r.text)


Status: 200
{"status":"completed"}


In [16]:
patient_id = "5e5202af-44ad-4994-9515-ba2fb3eb4dec"

headers = {
    "x-user-id": patient_id,
    "x-user-role": "patient"
}

r = requests.get(f"{BASE}/cases/patient/{patient_id}", headers=headers)
print(r.status_code)
print(r.json())


200
[{'id': '1774e42a-c1b2-4743-984a-934bd7e9a4fc', 'patient_id': '5e5202af-44ad-4994-9515-ba2fb3eb4dec', 'uploaded_at': '2025-11-30T14:51:07.187026+00:00', 'storage_path': 'pending', 'status': 'uploaded', 'updated_at': '2025-11-30T14:51:07.187026+00:00'}, {'id': '060b6466-8a5b-45ae-b348-5d0c6864c5d3', 'patient_id': '5e5202af-44ad-4994-9515-ba2fb3eb4dec', 'uploaded_at': '2025-11-30T14:53:52.696452+00:00', 'storage_path': 'pending', 'status': 'uploaded', 'updated_at': '2025-11-30T14:53:52.696452+00:00'}, {'id': '5097f192-e3f6-4514-8a77-9199983a750c', 'patient_id': '5e5202af-44ad-4994-9515-ba2fb3eb4dec', 'uploaded_at': '2025-11-30T15:02:57.222918+00:00', 'storage_path': '5097f192-e3f6-4514-8a77-9199983a750c/test_ct.zip', 'status': 'processing', 'updated_at': '2025-11-30T16:25:20.970227+00:00'}, {'id': '99d04c3e-c389-4dd3-9939-a78029f69886', 'patient_id': '5e5202af-44ad-4994-9515-ba2fb3eb4dec', 'uploaded_at': '2025-11-30T18:11:30.922211+00:00', 'storage_path': '99d04c3e-c389-4dd3-9939-a78

In [17]:
import requests

BASE = "http://127.0.0.1:8000"

case_id = "99d04c3e-c389-4dd3-9939-a78029f69886"
operator_id = "6255e461-2a14-4b3c-a635-de4428668fc0"

headers = {
    "x-user-id": operator_id,
    "x-user-role": "operator"
}

r = requests.post(f"{BASE}/process/case/{case_id}", headers=headers)
print("Status:", r.status_code)
print(r.json())


Status: 200
{'status': 'completed'}


In [18]:
patient_id = "5e5202af-44ad-4994-9515-ba2fb3eb4dec"

headers = {
    "x-user-id": patient_id,
    "x-user-role": "patient"
}

r = requests.get(f"{BASE}/cases/patient/{patient_id}", headers=headers)
print("Status:", r.status_code)
print(r.json())


Status: 200
[{'id': '1774e42a-c1b2-4743-984a-934bd7e9a4fc', 'patient_id': '5e5202af-44ad-4994-9515-ba2fb3eb4dec', 'uploaded_at': '2025-11-30T14:51:07.187026+00:00', 'storage_path': 'pending', 'status': 'uploaded', 'updated_at': '2025-11-30T14:51:07.187026+00:00'}, {'id': '060b6466-8a5b-45ae-b348-5d0c6864c5d3', 'patient_id': '5e5202af-44ad-4994-9515-ba2fb3eb4dec', 'uploaded_at': '2025-11-30T14:53:52.696452+00:00', 'storage_path': 'pending', 'status': 'uploaded', 'updated_at': '2025-11-30T14:53:52.696452+00:00'}, {'id': '5097f192-e3f6-4514-8a77-9199983a750c', 'patient_id': '5e5202af-44ad-4994-9515-ba2fb3eb4dec', 'uploaded_at': '2025-11-30T15:02:57.222918+00:00', 'storage_path': '5097f192-e3f6-4514-8a77-9199983a750c/test_ct.zip', 'status': 'processing', 'updated_at': '2025-11-30T16:25:20.970227+00:00'}, {'id': '99d04c3e-c389-4dd3-9939-a78029f69886', 'patient_id': '5e5202af-44ad-4994-9515-ba2fb3eb4dec', 'uploaded_at': '2025-11-30T18:11:30.922211+00:00', 'storage_path': '99d04c3e-c389-4dd3-

In [19]:
doctor_id = "358f00bb-d2f3-4352-b9bd-29042d3e9c57"

headers = {
    "x-user-id": doctor_id,
    "x-user-role": "doctor"
}

r = requests.get(f"{BASE}/cases/unassigned", headers=headers)
print("Status:", r.status_code)
print(r.json())


Status: 200
[]


In [20]:
case_id = "99d04c3e-c389-4dd3-9939-a78029f69886"

headers = {
    "x-user-id": doctor_id,
    "x-user-role": "doctor"
}

r = requests.post(f"{BASE}/doctor/accept/{case_id}", headers=headers)
print("Status:", r.status_code)
print(r.json())


Status: 200
{'status': 'assigned', 'assignment': {'id': 'd6326baf-feca-40f4-891b-81bf7d465d82', 'scan_id': '99d04c3e-c389-4dd3-9939-a78029f69886', 'doctor_id': '358f00bb-d2f3-4352-b9bd-29042d3e9c57', 'accepted_at': '2025-11-30T18:50:47.636607+00:00', 'status': 'assigned'}}


In [22]:
import requests

BASE = "http://127.0.0.1:8000"

doctor_id = "358f00bb-d2f3-4352-b9bd-29042d3e9c57"
case_id = "99d04c3e-c389-4dd3-9939-a78029f69886"  # example

headers = {
    "x-user-id": doctor_id,
    "x-user-role": "doctor"
}

r = requests.post(f"{BASE}/doctor/accept/{case_id}", headers=headers)
print("Status:", r.status_code)
print(r.json())


Status: 400
{'detail': 'Case already accepted: {\'code\': \'23505\', \'details\': \'Key (scan_id)=(99d04c3e-c389-4dd3-9939-a78029f69886) already exists.\', \'hint\': None, \'message\': \'duplicate key value violates unique constraint "doctor_assignments_scan_id_key"\'}'}


In [25]:
doctor_id = "358f00bb-d2f3-4352-b9bd-29042d3e9c57"
case_id    = "99d04c3e-c389-4dd3-9939-a78029f69886"


In [27]:
import requests

BASE = "http://127.0.0.1:8000"

headers = {
    "x-user-id": "358f00bb-d2f3-4352-b9bd-29042d3e9c57",
    "x-user-role": "doctor"
}

r = requests.get(f"{BASE}/doctor/assignment/{case_id}", headers=headers)
print("Status:", r.status_code)
print(r.json())


Status: 200
{'id': 'd6326baf-feca-40f4-891b-81bf7d465d82', 'scan_id': '99d04c3e-c389-4dd3-9939-a78029f69886', 'doctor_id': '358f00bb-d2f3-4352-b9bd-29042d3e9c57', 'accepted_at': '2025-11-30T18:50:47.636607+00:00', 'status': 'assigned'}


In [31]:
assignment_id = "d6326baf-feca-40f4-891b-81bf7d465d82"

headers = {
    "x-user-id": "358f00bb-d2f3-4352-b9bd-29042d3e9c57",
    "x-user-role": "doctor"
}

payload = {"message": "Hello, patient. I have reviewed your CT scan."}

r = requests.post(f"{BASE}/chat/send/{assignment_id}", headers=headers, json=payload)
print(r.status_code, r.json())


200 {'id': 'c3f9bec2-d0e2-4e52-915b-c971d28e12ff', 'assignment_id': 'd6326baf-feca-40f4-891b-81bf7d465d82', 'sender_id': '358f00bb-d2f3-4352-b9bd-29042d3e9c57', 'message': 'Hello, patient. I have reviewed your CT scan.', 'attachment_url': None, 'sent_at': '2025-11-30T19:20:05.124541+00:00'}


In [33]:
assignment_id = "d6326baf-feca-40f4-891b-81bf7d465d82"

headers = {
    "x-user-id": "5e5202af-44ad-4994-9515-ba2fb3eb4dec",  # patient
    "x-user-role": "patient"
}

r = requests.get(f"{BASE}/chat/history/{assignment_id}", headers=headers)
print("Status:", r.status_code)
print(r.json())


Status: 200
[{'id': 'c3f9bec2-d0e2-4e52-915b-c971d28e12ff', 'assignment_id': 'd6326baf-feca-40f4-891b-81bf7d465d82', 'sender_id': '358f00bb-d2f3-4352-b9bd-29042d3e9c57', 'message': 'Hello, patient. I have reviewed your CT scan.', 'attachment_url': None, 'sent_at': '2025-11-30T19:20:05.124541+00:00'}]


In [34]:
import requests

BASE = "http://127.0.0.1:8000"

patient_id = "5e5202af-44ad-4994-9515-ba2fb3eb4dec"
assignment_id = "d6326baf-feca-40f4-891b-81bf7d465d82"

headers = {
    "x-user-id": patient_id,
    "x-user-role": "patient"
}

r = requests.get(f"{BASE}/chat/history/{assignment_id}", headers=headers)
print(r.status_code)
print(r.json())


200
[{'id': 'c3f9bec2-d0e2-4e52-915b-c971d28e12ff', 'assignment_id': 'd6326baf-feca-40f4-891b-81bf7d465d82', 'sender_id': '358f00bb-d2f3-4352-b9bd-29042d3e9c57', 'message': 'Hello, patient. I have reviewed your CT scan.', 'attachment_url': None, 'sent_at': '2025-11-30T19:20:05.124541+00:00'}]


In [35]:
doctor_id = "358f00bb-d2f3-4352-b9bd-29042d3e9c57"

headers = {
    "x-user-id": doctor_id,
    "x-user-role": "doctor"
}

payload = {"message": "Please drink more water."}

r = requests.post(
    f"{BASE}/chat/send/{assignment_id}",
    headers=headers,
    json=payload
)

print(r.status_code, r.json())


200 {'id': '9b1d8b29-e794-41e9-9a96-66842e6fa447', 'assignment_id': 'd6326baf-feca-40f4-891b-81bf7d465d82', 'sender_id': '358f00bb-d2f3-4352-b9bd-29042d3e9c57', 'message': 'Please drink more water.', 'attachment_url': None, 'sent_at': '2025-11-30T19:32:34.666245+00:00'}


In [36]:
headers = {
    "x-user-id": patient_id,
    "x-user-role": "patient"
}

payload = {"message": "Thank you doctor!"}

r = requests.post(
    f"{BASE}/chat/send/{assignment_id}",
    headers=headers,
    json=payload
)

print(r.status_code, r.json())


200 {'id': '0ca99eb0-ac59-4541-8bfc-991082fea2dc', 'assignment_id': 'd6326baf-feca-40f4-891b-81bf7d465d82', 'sender_id': '5e5202af-44ad-4994-9515-ba2fb3eb4dec', 'message': 'Thank you doctor!', 'attachment_url': None, 'sent_at': '2025-11-30T19:32:42.075115+00:00'}


In [37]:
headers = {
    "x-user-id": patient_id,
    "x-user-role": "patient"
}

r = requests.get(f"{BASE}/chat/history/{assignment_id}", headers=headers)
print(r.status_code)
print(r.json())


200
[{'id': 'c3f9bec2-d0e2-4e52-915b-c971d28e12ff', 'assignment_id': 'd6326baf-feca-40f4-891b-81bf7d465d82', 'sender_id': '358f00bb-d2f3-4352-b9bd-29042d3e9c57', 'message': 'Hello, patient. I have reviewed your CT scan.', 'attachment_url': None, 'sent_at': '2025-11-30T19:20:05.124541+00:00'}, {'id': '9b1d8b29-e794-41e9-9a96-66842e6fa447', 'assignment_id': 'd6326baf-feca-40f4-891b-81bf7d465d82', 'sender_id': '358f00bb-d2f3-4352-b9bd-29042d3e9c57', 'message': 'Please drink more water.', 'attachment_url': None, 'sent_at': '2025-11-30T19:32:34.666245+00:00'}, {'id': '0ca99eb0-ac59-4541-8bfc-991082fea2dc', 'assignment_id': 'd6326baf-feca-40f4-891b-81bf7d465d82', 'sender_id': '5e5202af-44ad-4994-9515-ba2fb3eb4dec', 'message': 'Thank you doctor!', 'attachment_url': None, 'sent_at': '2025-11-30T19:32:42.075115+00:00'}]


In [5]:
import requests

BASE = "http://127.0.0.1:8000"

patient_id = "5e5202af-44ad-4994-9515-ba2fb3eb4dec"
operator_id = "6255e461-2a14-4b3c-a635-de4428668fc0"
doctor_id = "358f00bb-d2f3-4352-b9bd-29042d3e9c57"

# ---------------------------------------
# STEP 1 — UPLOAD ZIP AS PATIENT
# ---------------------------------------
files = {"file": open("test_ct_extracted.zip", "rb")}
headers_patient = {"x-user-id": patient_id, "x-user-role": "patient"}

r = requests.post(f"{BASE}/upload/scan", headers=headers_patient, files=files)
print("UPLOAD:", r.status_code, r.json())

case_id = r.json()["case_id"]


# ---------------------------------------
# STEP 2 — OPERATOR PROCESS CASE
# ---------------------------------------
headers_operator = {"x-user-id": operator_id, "x-user-role": "operator"}

r = requests.post(f"{BASE}/process/case/{case_id}", headers=headers_operator)
print("PROCESS:", r.status_code, r.text)


# ---------------------------------------
# STEP 3 — CHECK ml_json BUCKET VIA DB
# ---------------------------------------
headers_patient = {"x-user-id": patient_id, "x-user-role": "patient"}

r = requests.get(f"{BASE}/cases/{case_id}", headers=headers_patient)
print("CASE:", r.status_code, r.json())

# CASE OUTPUT SHOULD SHOW:
# "json_path": "case_id/findings.json"


UPLOAD: 200 {'case_id': '9bd6694d-03ba-46ed-9637-18d7b0a2be53', 'storage_path': '9bd6694d-03ba-46ed-9637-18d7b0a2be53/test_ct_extracted.zip'}
PROCESS: 500 {"detail":"Pipeline error: Pipeline did not produce findings.json"}
CASE: 200 {'id': '9bd6694d-03ba-46ed-9637-18d7b0a2be53', 'patient_id': '5e5202af-44ad-4994-9515-ba2fb3eb4dec', 'uploaded_at': '2025-11-30T21:53:29.759283+00:00', 'storage_path': '9bd6694d-03ba-46ed-9637-18d7b0a2be53/test_ct_extracted.zip', 'status': 'failed', 'updated_at': '2025-11-30T21:53:49.711539+00:00'}


In [13]:
import requests
import time
import json

BASE = "http://127.0.0.1:8000"

# ---------------------------------------------------------
# CONFIGURE USER IDS
# ---------------------------------------------------------
PATIENT_ID = "5e5202af-44ad-4994-9515-ba2fb3eb4dec"
OPERATOR_ID = "6255e461-2a14-4b3c-a635-de4428668fc0"
DOCTOR_ID = "358f00bb-d2f3-4352-b9bd-29042d3e9c57"

ZIP_FILE = "test_ct_extracted.zip"   # <== set your test ZIP file here


# ---------------------------------------------------------
def pp(title, res):
    print(f"\n=== {title} ===")
    print("Status:", res.status_code)
    try:
        print(json.dumps(res.json(), indent=2))
    except:
        print(res.text)


# ---------------------------------------------------------
# 1) PATIENT UPLOADS CT ZIP
# ---------------------------------------------------------
files = {"file": open(ZIP_FILE, "rb")}
headers = {"x-user-id": PATIENT_ID, "x-user-role": "patient"}

res = requests.post(f"{BASE}/upload/scan", headers=headers, files=files)
pp("UPLOAD", res)

case_id = res.json()["case_id"]
print("\nCase ID:", case_id)

print("\nRunning pipeline... (This may take 30–120 seconds)")
time.sleep(2)


# ---------------------------------------------------------
# 2) OPERATOR PROCESSES SCAN
# ---------------------------------------------------------
headers = {"x-user-id": OPERATOR_ID, "x-user-role": "operator"}

res = requests.post(f"{BASE}/process/case/{case_id}", headers=headers)
pp("PROCESS", res)


# ---------------------------------------------------------
# 3) CHECK CASE DETAILS
# ---------------------------------------------------------
headers = {"x-user-id": PATIENT_ID, "x-user-role": "patient"}

res = requests.get(f"{BASE}/cases/{case_id}", headers=headers)
pp("CASE DETAILS", res)


# ---------------------------------------------------------
# 4) CHECK FINDINGS JSON (scan_results table)
# ---------------------------------------------------------
res = requests.get(f"{BASE}/scan_results/{case_id}", headers=headers)
pp("SCAN RESULTS", res)


# ---------------------------------------------------------
# 5) DOCTOR FETCHES UNASSIGNED CASES
# ---------------------------------------------------------
headers = {"x-user-id": DOCTOR_ID, "x-user-role": "doctor"}

res = requests.get(f"{BASE}/cases/unassigned", headers=headers)
pp("UNASSIGNED CASES", res)


# ---------------------------------------------------------
# 6) DOCTOR ACCEPTS CASE
# ---------------------------------------------------------
res = requests.post(f"{BASE}/doctor/accept/{case_id}", headers=headers)
pp("DOCTOR ACCEPTS CASE", res)

assignment_id = res.json().get("id")   # depends on API return
print("\nAssignment ID:", assignment_id)


# ---------------------------------------------------------
# 7) DOCTOR SENDS CHAT MESSAGE
# ---------------------------------------------------------
payload = {"message": "Your CT scan report is ready. Please check your results."}

res = requests.post(
    f"{BASE}/chat/send/{assignment_id}",
    headers=headers,
    json=payload
)
pp("DOCTOR SENDS MESSAGE", res)


# ---------------------------------------------------------
# 8) PATIENT FETCHES CHAT HISTORY
# ---------------------------------------------------------
headers = {"x-user-id": PATIENT_ID, "x-user-role": "patient"}

res = requests.get(f"{BASE}/chat/history/{assignment_id}", headers=headers)
pp("CHAT HISTORY", res)



=== UPLOAD ===
Status: 200
{
  "case_id": "5a4f4c70-db98-4ea5-ba84-e6d62776d285",
  "storage_path": "5a4f4c70-db98-4ea5-ba84-e6d62776d285/test_ct_extracted.zip"
}

Case ID: 5a4f4c70-db98-4ea5-ba84-e6d62776d285

Running pipeline... (This may take 30–120 seconds)

=== PROCESS ===
Status: 200
{
  "status": "completed"
}

=== CASE DETAILS ===
Status: 200
{
  "id": "5a4f4c70-db98-4ea5-ba84-e6d62776d285",
  "patient_id": "5e5202af-44ad-4994-9515-ba2fb3eb4dec",
  "uploaded_at": "2025-11-30T22:37:17.973716+00:00",
  "storage_path": "5a4f4c70-db98-4ea5-ba84-e6d62776d285/test_ct_extracted.zip",
  "status": "processing",
  "updated_at": "2025-11-30T22:37:26.729208+00:00"
}

=== SCAN RESULTS ===
Status: 404
{
  "detail": "Not Found"
}

=== UNASSIGNED CASES ===
Status: 200
[]

=== DOCTOR ACCEPTS CASE ===
Status: 200
{
  "id": "cdbdc14b-3131-44a9-9c00-16455aa38dc0",
  "scan_id": "5a4f4c70-db98-4ea5-ba84-e6d62776d285",
  "doctor_id": "358f00bb-d2f3-4352-b9bd-29042d3e9c57",
  "status": "assigned",
  